In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('whaleloops/phrase-bert')

In [ ]:
import torch
from torch import nn
# place ground-truth sentence at phrase_list[0], distractor at phrase_list[1]
def check_cos_sim(model, phrase_list, threshold):
    phrase_embs = model.encode(phrase_list)
    [gt, dt, op1, op2, op3] = phrase_embs
    cos_sim = nn.CosineSimilarity(dim=0)
    all_distant = True
    # closeness = []
    cos_sim_list = []
    cos_sim_list.append(cos_sim(torch.tensor(gt), torch.tensor(dt)))
    cos_sim_list.append(cos_sim(torch.tensor(gt), torch.tensor(op1)))
    cos_sim_list.append(cos_sim(torch.tensor(gt), torch.tensor(op2)))
    cos_sim_list.append(cos_sim(torch.tensor(gt), torch.tensor(op3)))

    cos_sim_list.append(cos_sim(torch.tensor(dt), torch.tensor(op1)))
    cos_sim_list.append(cos_sim(torch.tensor(dt), torch.tensor(op2)))
    cos_sim_list.append(cos_sim(torch.tensor(dt), torch.tensor(op3)))

    for i in range(len(cos_sim_list)):
        if cos_sim_list[i] > threshold:
            # closeness.append((i//4, (i//4)+(i%4)+1))
            all_distant = False
            print(f"[{phrase_list[i//4]}] -  [{phrase_list[(i//4)+(i%4)+1]}]: {cos_sim_list[i]}; Current threshold is {threshold}\n")
    return all_distant

In [ ]:
import json
with open("./old_verb_answers_full.json", 'r') as f:
    answer_list = json.load(f)

In [ ]:
filtered_answer_list = []

for answer in answer_list:
    ppl_dict = answer['ppl']
    ppl_list = list(ppl_dict.keys())
    ppl_list[0], ppl_list[1] = ppl_list[1], ppl_list[0]
    if check_cos_sim(model, ppl_list, 0.71):
        filtered_answer_list.append(answer)

In [ ]:
len(filtered_answer_list)

In [ ]:
a = "Where is a ghost cooking a meal?\nA. up the hill\nB. in the refrigerator\nC. around a lake\nD. up the tree\nE. in school"
a.split('\n')[1:]

In [ ]:
error = {}
choice_success = {}
unexpected = {}
option_list = ['A', 'B', 'C', 'D', 'E']
binary_list = ['Yes', 'No']
for answer in filtered_answer_list:
    q_id = answer['id']
    ppl_list = answer['ppl']
    distractor = min(ppl_list, key=ppl_list.get)
    options = answer['choice'].split('\n')[1:]
    dt_option = None
    for i in range(len(options)):
        if distractor in options[i]:
            dt_option = option_list[i]
            break
    gt_option = answer['choice answer']
    yes_ans = answer['binary-yes answer'].split('.')[0]
    no_ans = answer['binary-no answer'].split('.')[0]
    cp_ans = answer['binary-cp answer'].split('.')[0]

    local_error = {"choice":[], "binary-yes":[], "binary-no":[], "binary-cp":[]}
    local_unexp = {"choice":[], "binary-yes":[], "binary-no":[], "binary-cp":[]}
    local_choice_success = []
    for i in range(2):
        for j in range(5):
            choice_key = f"choice_ans_img{i+1}_{j+1}"
            choice_answer_text = answer[choice_key]
            if not choice_answer_text.startswith(gt_option):
                if choice_answer_text.startswith(dt_option):
                    local_choice_success.append(choice_key)
                    local_error['choice'].append(choice_key)
                else:
                    not_start_letter = True
                    for letter in option_list:
                        if choice_answer_text.startswith(letter):
                            not_start_letter = False
                    if not_start_letter:
                        local_unexp['choice'].append(choice_key)
                    else:
                        local_error['choice'].append(choice_key)

            binary_yes_key = f"binary-yes_ans_img{i+1}_{j+1}"
            binary_yes_answer = answer[binary_yes_key]
            if not binary_yes_answer.startswith(yes_ans):
                not_start_y_n = True
                for binary in binary_list:
                    if binary_yes_answer.startswith(binary):
                        not_start_y_n = False
                        break
                if not_start_y_n:
                    local_unexp['binary-yes'].append(binary_yes_key)
                else:
                    local_error['binary-yes'].append(binary_yes_key)

            binary_no_key = f"binary-no_ans_img{i+1}_{j+1}"
            binary_no_answer = answer[binary_no_key]
            if not binary_no_answer.startswith(no_ans):
                not_start_y_n = True
                for binary in binary_list:
                    if binary_no_answer.startswith(binary):
                        not_start_y_n = False
                        break
                if not_start_y_n:
                    local_unexp['binary-no'].append(binary_no_key)
                else:
                    local_error['binary-no'].append(binary_no_key)

            binary_cp_key = f"binary-cp_ans_img{i+1}_{j+1}"
            # binary_cp_key = f"binary-cp_img{i+1}_{j+1}"
            binary_cp_answer = answer[binary_cp_key]
            if not binary_cp_answer.startswith(cp_ans):
                not_start_y_n = True
                for binary in binary_list:
                    if binary_cp_answer.startswith(binary):
                        not_start_y_n = False
                        break
                if not_start_y_n:
                    local_unexp['binary-cp'].append(binary_cp_key)
                else:
                    local_error['binary-cp'].append(binary_cp_key)

    error[q_id] = local_error
    unexpected[q_id] = local_unexp
    choice_success[q_id] = local_choice_success

In [ ]:
choice_error = {}
for qid, error_list in error.items():
    if len(error_list['choice']) != 0:
        choice_error[qid] = error_list['choice']
# choice_error

In [ ]:
short_list_choice = []
for id, key_list in choice_error.items():
    img1, img2 = 0, 0
    for key_name in key_list:
        if "img1" in key_name:
            img1 += 1
        elif "img2" in key_name:
            img2 += 1
    if img1 != 0:
        short_list_choice.append(f"{str(id)}a({img1})")
    if img2 != 0:
        short_list_choice.append(f"{str(id)}b({img2})")

for i in short_list_choice:
    print(i, end=' ')

In [ ]:
choice_success_short = []
for id, key_list in choice_success.items():
    if len(key_list) != 0:
        img1, img2 = 0, 0
        for key_name in key_list:
            if "img1" in key_name:
                img1 += 1
            elif "img2" in key_name:
                img2 += 1
        if img1 != 0:
            choice_success_short.append(f"{str(id)}a({img1})")
        if img2 != 0:
            choice_success_short.append(f"{str(id)}b({img2})")

for i in choice_success_short:
    print(i, end=', ')

In [ ]:
binary_yes_error = {}
for qid, error_list in error.items():
    if len(error_list['binary-yes']) != 0:
        binary_yes_error[qid] = error_list['binary-yes']

short_list_yes = []
for id, key_list in binary_yes_error.items():
    img1, img2 = 0, 0
    for key_name in key_list:
        if "img1" in key_name:
            img1 += 1
        elif "img2" in key_name:
            img2 += 1
    if img1 != 0:
        short_list_yes.append(f"{str(id)}a({img1})")
    if img2 != 0:
        short_list_yes.append(f"{str(id)}b({img2})")

for i in short_list_yes:
    print(i, end=' ')

In [ ]:
binary_no_error = {}
for qid, error_list in error.items():
    if len(error_list['binary-no']) != 0:
        binary_no_error[qid] = error_list['binary-no']
binary_no_error

In [ ]:
short_list_no = []
for id, key_list in binary_no_error.items():
    img1, img2 = 0, 0
    for key_name in key_list:
        if "img1" in key_name:
            img1 += 1
        elif "img2" in key_name:
            img2 += 1
    if img1 != 0:
        short_list_no.append(f"{str(id)}a({img1})")
    if img2 != 0:
        short_list_no.append(f"{str(id)}b({img2})")

for i in short_list_no:
    print(i, end=' ')

In [ ]:
binary_cp_error = {}
for qid, error_list in error.items():
    if len(error_list['binary-cp']) != 0:
        binary_cp_error[qid] = error_list['binary-cp']
binary_cp_error

In [ ]:
short_list_cp = []
for id, key_list in binary_cp_error.items():
    img1, img2 = 0, 0
    for key_name in key_list:
        if "img1" in key_name:
            img1 += 1
        elif "img2" in key_name:
            img2 += 1
    if img1 != 0:
        short_list_cp.append(f"{str(id)}a({img1})")
    if img2 != 0:
        short_list_cp.append(f"{str(id)}b({img2})")

for i in short_list_cp:
    print(i, end=' ')